In [ ]:
import numpy as np
import sys
import os
from os import path
import argparse
import math
sys.path.insert(0, "/home/hongzl/works/caffe/python/")
import caffe

def ConvertToFullConvNet():
    net0 = "VGG_ILSVRC_16_layers_deploy.prototxt"
    model0 = "VGG_ILSVRC_16_layers.caffemodel"
    net1 = "VGG_ILSVRC_16_layers_deploy_bp.prototxt"
    model1 = "VGG_ILSVRC_16_layers_bp.caffemodel"
    model0_copy = "VGG_ILSVRC_16_layers_copy.caffemodel"
    # Load the original network and extract the fully connected layers' parameters.
    net = caffe.Net(net0, model0, caffe.TEST)
    
    params = net.params.keys()[-3:]
    print 'layers that will be converted: {}'.format(params)
    # fc_params = {name: (weights, biases)}
    fc_params = {pr: (net.params[pr][0].data, net.params[pr][1].data) for pr in params}

    for fc in params:
        print '{} weights are {} dimensional and biases are {} dimensional'.format(fc, fc_params[fc][0].shape, fc_params[fc][1].shape)

    # Load the fully convolutional network to transplant the parameters.
    net_full_conv = caffe.Net(net1, model0, caffe.TEST)
    params_full_conv = net_full_conv.params.keys()[-3:]
    # conv_params = {name: (weights, biases)}
    conv_params = {pr: (net_full_conv.params[pr][0].data, net_full_conv.params[pr][1].data) for pr in params_full_conv}

    for conv in params_full_conv:
        print '{} weights are {} dimensional and biases are {} dimensional'.format(conv, conv_params[conv][0].shape, conv_params[conv][1].shape)

    for pr, pr_conv in zip(params, params_full_conv):
        conv_params[pr_conv][0].flat = fc_params[pr][0].flat  # flat unrolls the arrays
        conv_params[pr_conv][1][...] = fc_params[pr][1]

    net_full_conv.save(model1)
    net.save(model0_copy)

ConvertToFullConvNet()